In [2]:
!pip install altair==4.2.0
import altair as alt
import pandas as pd
import numpy as np
from collections import Counter
import datetime as dt
RANDOM_SEED=697

     |████████████████████████████████| 812 kB 34.3 MB/s 
  Attempting uninstall: altair
    Found existing installation: altair 4.1.0
    Not uninstalling altair at /shared-libs/python3.7/py/lib/python3.7/site-packages, outside environment /root/venv
    Can't uninstall 'altair'. No files were found to uninstall.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
# alt.data_transformers.disable_max_rows()
alt.renderers.enable('default')

RendererRegistry.enable('default')

In [5]:
brightorange='#ffa500'
Orange='#FFCC00'
Yellow = '#FFE680'
grey3 = '#343852'
redorange='#E55B13'
Lightgrey = '#D3D3D3'
pewter = '#899499'
slate = '#708090'
Darkgrey = '#A9A9A9'

In [13]:
#------------------------------------------------------
# # re-generating this data requires the whole dataset
# please request through github
# not necessary to generate visualizations below
#------------------------------------------------------
# c = pd.read_csv('data/processed/comments.csv', parse_dates=['created', 'retrieved'])

# c['created_day'] = c.created.dt.day
# c_day = c.groupby(['created_day', 'gender'], as_index=False).count()
# c_day.to_csv('data/viz/comments_summary_gender.csv', index=False)

### 1. Bursting Sun - summary 

In [14]:
source = pd.read_csv('data/viz/comments_summary_gender.csv')

selection = alt.selection_multi(fields=['gender'], bind='legend')

base = alt.Chart(source.reset_index()).encode(
    theta=alt.Theta('index:O', stack=True, band=0.6),
    radius=alt.Radius('status:Q', scale=alt.Scale(type="sqrt", zero=True, rangeMin=20)),
    color=alt.Color('gender', 
                    scale=alt.Scale(range=[Yellow, Orange, redorange, brightorange]),
                    legend=alt.Legend( 
                    symbolType='circle', 
                    symbolSize=200)),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    tooltip=[
        alt.Tooltip('status', title='count'),
        alt.Tooltip('created_day', title='day'),
        alt.Tooltip('gender', title='gender')])
c1 = base.mark_arc(innerRadius=50, cornerRadius=3).add_selection(selection)

c1.properties(width=500, height=500, title='Primary dataset daily gender count'
).configure_view(stroke=None
).configure_title(
    fontSize=20,
    font='Nunito',
    anchor='middle',
    dy=-15,
    color=grey3
).configure_legend(
    padding=10,
    cornerRadius=10,
    labelFont='Nunito',
    titleFont='Nunito Light'
)

alt.Chart(...)

In [16]:
#------------------------------------------------------
# # re-generating this data requires the whole dataset
# please request through github
# not necessary to generate visualizations below
#------------------------------------------------------

# c_davidson_summary = pd.DataFrame(c.davidson_label.value_counts()).rename(columns={'davidson_label': 'num_comments'})
# c_davidson_summary['lang_class'] = c_davidson_summary.index.map({0:'hateful', 1:'offensive', 2:'neither'})
# c_davidson_summary.to_csv('data/viz/comments_davidson_summary.csv', index=False)

In [17]:
source = pd.read_csv('data/viz/comments_davidson_summary.csv')

source['lang_class'] = ['2: neither','1: offensive','0: hateful']
bar = alt.Chart(source).mark_bar(size=40).encode(
    alt.X('num_comments', 
            axis=None, 
            stack='zero'),
    alt.Color('lang_class:N', 
                scale=alt.Scale(range=[Orange, Yellow, Lightgrey]),
                legend=alt.Legend( 
                    symbolType='circle', 
                    title=None,
                    orient='bottom',
                    symbolSize=200)
                ),
)
txt = bar.mark_text(dx=-40, dy=3, font='Nunito').encode(
    alt.Text('num_comments'),
    alt.ColorValue('black')
)

(bar+txt
).configure_view(stroke=None
).properties(width=800
).configure_legend(
    padding=10,
    cornerRadius=10,
    labelFont='Nunito',
    titleFont='Nunito Light'
)

alt.LayerChart(...)

In [18]:
# ref = pd.read_csv('data/processed/reference.csv', parse_dates=['created'])

# ref['created_day'] = ref.created.dt.day
# ref_day = ref.groupby(['created_day', 'gender'], as_index=False).count()
# ref_day.to_csv('data/viz/ref_summary_gender.csv', index=False)

# c_gender = pd.DataFrame(c.gender.value_counts())
# r_gender = pd.DataFrame(ref.gender.value_counts())
# gender_compare = c_gender.join(r_gender,lsuffix='_comments', rsuffix='_reference')
# gender_compare['primary'] = (gender_compare.gender_comments/gender_compare.gender_comments.sum()).round(3)
# gender_compare['reference'] = (gender_compare.gender_reference/gender_compare.gender_reference.sum()).round(3)
# gender_compare = (gender_compare
#                 .reset_index()
#                 .melt(id_vars='index', value_vars=['primary', 'reference'])
#                 .rename(columns={'index':'gender', 'variable':'dataset', 'value': 'percentage'}))

# gender_compare.to_csv('data/viz/gender_compare.csv', index=False)

In [19]:
source = pd.read_csv('data/viz/gender_compare.csv')

alt.Chart(source).mark_bar(cornerRadius=1.5).encode(
    alt.X('percentage:Q', title=None),
    alt.Y('dataset', title=None, axis=None),
    alt.Row('gender', spacing=10, 
            title=None,
            header=alt.Header(
                labelAngle=0,
                labelAlign='left',
                labelPadding=20,
                labelFontWeight=500
                )
            ),
    alt.Color('dataset', 
                scale=alt.Scale(range=[Yellow, Lightgrey]),
                legend=alt.Legend( 
                    symbolType='circle', 
                    titleFontWeight=500,
                    symbolSize=200)
                    ),
    alt.Tooltip('percentage')
).configure_axis(grid=False
).configure_view(stroke=None
).properties(
    width=200, title='Gender distribution comparison between primary and reference'
).configure_title(
    fontSize=18,
    font='Nunito',
    anchor='start',
    dy=-15,
    color=grey3
)

alt.Chart(...)

#### New chart to show median of detoxify scores per gender

In [21]:
detox_gender = pd.read_csv('data/viz/RQ2_gender_median.csv')
detox_gender = detox_gender.loc[:, ['gender', 'toxicity', 'severe_toxicity',
       'obscene', 'identity_attack', 'insult', 'threat', 'sexual_explicit']]
detox_gender = detox_gender.melt(value_vars=detox_gender.columns.to_list()[1:], 
                                        id_vars='gender', 
                                        var_name='score_type', 
                                        value_name='median_score')

detox_gender.to_csv('data/viz/detox_gender.csv', index=False)

In [22]:
source = pd.read_csv('data/viz/detox_gender.csv')

alt.Chart(source.round(3)).mark_bar(cornerRadius=1.5).encode(
    alt.Y('gender:N', title=None, axis=alt.Axis(labels=False, ticks=False)),
    alt.X('median_score', title=None, axis=alt.Axis(grid=False)),
    alt.Row('score_type', 
            title=None, 
            header=alt.Header(
                            labelAngle=0,
                            labelAlign='left',
                            labelPadding=20,
                            labelFontWeight=500
                            )),
    alt.Color('gender:N', title=None, 
                scale=alt.Scale(range=[Orange, Yellow, brightorange, redorange]),
                legend=alt.Legend( 
                    symbolType='circle', 
                    symbolSize=200)
                ),
    tooltip=[alt.Tooltip('median_score', title='median score'),
            alt.Tooltip('gender')]
).properties(height=50, width=200, title="Median detoxify scores per gender"
).configure_view(stroke=None
).configure_title(
    fontSize=18,
    font='Nunito',
    anchor='start',
    dy=-15,
    color=grey3
).configure_legend(
    padding=10,
    cornerRadius=10,
    labelFont='Nunito',
    titleFont='Nunito Light'
)

alt.Chart(...)

#### Median detoxify per davidson_label

In [23]:
detox_hate = pd.read_csv('data/viz/RQ17_davidson_median.csv')

source = detox_hate.melt(value_vars=detox_hate.columns.to_list()[1:], 
                                        id_vars='lang_class', 
                                        var_name='score_type', 
                                        value_name='median_score').round(3)

alt.Chart(source).mark_bar(cornerRadius=1.5, opacity=0.7).encode(
    alt.Y('lang_class:N', title=None, axis=alt.Axis(labels=False, ticks=False)),
    alt.X('median_score', axis=alt.Axis(grid=False), title=None),
    alt.Row('score_type', 
            title=None, 
            header=alt.Header(
                            labelAngle=0,
                            labelAlign='left',
                            labelPadding=10)),
    alt.Color('lang_class:N', 
                scale=alt.Scale(range=[Orange, pewter, brightorange]), 
                legend=alt.Legend(
                    title=None, 
                    symbolType='circle', 
                    symbolSize=200)),
    tooltip='median_score'
).properties(height=40, width=200, title="Median detoxify scores for davidson_label"
).configure_view(stroke=None
).configure_title(
    fontSize=18,
    font='Nunito',
    anchor='start',
    dy=-15,
    color=grey3
).configure_legend(
    padding=10,
    cornerRadius=10,
    labelFont='Nunito',
    titleFont='Nunito Light'
)

alt.Chart(...)

#### Top Tokens for comments classified as hateful or offensive

In [25]:
source = pd.read_csv('data/viz/top_tokens_hateful.csv')

rect = alt.Chart(source).mark_rect(opacity=0.8, stroke='white').encode(
    alt.X('gender', title=None, axis=alt.Axis(orient='top',
                                            labelAngle=0, 
                                            ticks=False, 
                                            labelFont='Nunito', 
                                            labelPadding=5)),
    alt.Y('rank:O', title=None, axis=None),
    alt.Color('gender', 
                scale=alt.Scale(range=[Yellow, Orange, brightorange]),
                legend=None)
).properties(width = 400)

txt = rect.mark_text(font='Nunito', fontWeight=300, fontSize=12
).encode(
    alt.Text('token'),
    alt.ColorValue('black'),
    tooltip=[alt.Tooltip('freq', title='frequency')]
)

(rect+txt
).properties(title={'text':'Top tokens for hateful comments',
                'subtitle':['(comments classified as either hateful or offensive)', ' '],
                'subtitleFontSize': 12} 
).configure_title(
    fontSize=18,
    font='Nunito',
    anchor='middle',
    dy=-15,
    color=grey3
).configure_legend(
    padding=10,
    cornerRadius=10,
    labelFont='Nunito',
    titleFont='Nunito Light'
)

alt.LayerChart(...)

In [26]:
source = pd.read_csv('data/viz/top_tokens_reference.csv')

rect = alt.Chart(source).mark_rect(opacity=.6, stroke='white').encode(
    alt.X('gender', title=None, axis=alt.Axis(orient='top', 
                                            labelAngle=0, 
                                            ticks=False, 
                                            labelFont='Nunito', 
                                            labelPadding=5)),
    alt.Y('rank:O', title=None, axis=None),
    alt.Color('gender', scale=alt.Scale(range=[Lightgrey, pewter, slate]), legend=None)
).properties(width = 400)

txt = rect.mark_text(font='Nunito', fontWeight=300, fontSize=12
).encode(
    alt.Text('token'),
    alt.ColorValue('black'),
    tooltip=[alt.Tooltip('freq', title='frequency')]
)

(rect+txt
).properties(title='Top tokens per gender for reference dataset' 
).configure_title(
    fontSize=18,
    font='Nunito',
    anchor='middle',
    dy=-15,
    color=grey3
).configure_legend(
    padding=10,
    cornerRadius=10,
    labelFont='Nunito',
    titleFont='Nunito Light'
)

alt.LayerChart(...)

In [27]:
def get_bars(source, color):
    rect = alt.Chart(source
    ).mark_bar(color=color, 
    ).encode(
        alt.X('percentage', title=None, axis=None),
        alt.Y('rank:O', title=None, axis=None),
        tooltip=[alt.Tooltip('percentage:Q', title='% of this gender')]
    ).properties(width = 100)
    txt = rect.mark_text(align = 'left', dx=3, font='Nunito', fontWeight=300, fontSize=12
    ).encode(alt.Text('token'), alt.ColorValue('black'))
    chart = (rect+txt)

    return chart 

In [28]:
perc_id_hate = pd.read_csv('data/viz/top_gender_id_hate.csv')

f_data = perc_id_hate[perc_id_hate.gender=='f_terms']
m_data = perc_id_hate[perc_id_hate.gender=='m_terms']

hate_gender_id_bars = (get_bars(f_data, color=Yellow)|get_bars(m_data, color=Orange))

In [29]:
perc_id_ref = pd.read_csv('data/viz/top_gender_id_ref.csv')

f_data = perc_id_ref[perc_id_ref.gender=='f_terms']
m_data = perc_id_ref[perc_id_ref.gender=='m_terms']

ref_gender_is_bars = (get_bars(f_data, color=Lightgrey)|get_bars(m_data, color=pewter))


In [30]:
alt.hconcat(hate_gender_id_bars, ref_gender_is_bars, spacing=100
).properties(
    title= {'text': 'Top gender identifiers',
    'fontSize':20,
            'subtitle': [' ', 
            'Primary dataset                                Reference datase'],
            'subtitleFontSize':16,
            }
).configure_title(
        fontSize=18,
        font='Nunito',
        anchor='middle',
        dy=-5,
        color=grey3).configure_view(stroke=None
        ).configure_legend(padding=10, cornerRadius=10,
                                        labelFont='Nunito',titleFont='Nunito Light')

alt.HConcatChart(...)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b22dad3f-c925-4cd0-bb81-e22d83bd774f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>